# LDA topic modeling with a curated dataset

This example uses the [`gensim`](https://radimrehurek.com/gensim/index.html) library for building the topic model.

Depending on your collection size, this example will take between 10 minutes to an hour+ to run. 

In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
import gensim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

stop_words = set(stopwords.words('english'))

Initialize a dataset object. 

In [3]:
from tdm_core.client import Dataset

dset = Dataset('bb3d938b-bc61-4c2c-a21c-9a4f102035c8')

In [4]:
len(dset)

61

In [5]:
dset.query()

'q=%22walt%20whitman%22%20brooklyn&fq=yearPublished%3A%5B1700%20TO%202019%5D&fq=outputFormat%3Aunigrams'

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [6]:
def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the volumes in the dataset and make a list of tokens for each volume and then add to a list of all documents in the dataset.

In [7]:
dset_items = [d for d in dset]
documents = []    

In [9]:
for doc_n, volume in enumerate(dset_items):
    this_doc = []
    try:
        pages = volume['features']['pages']
    except:
        continue
    for pn, page in enumerate(pages):
        body = page.get('body')
        if body is not None:
            for token, pos_count in body.get('tokenPosCount', {}).items():
                t = process_token(token)
                if t is None:
                    continue
                for pos, n in pos_count.items():
                    this_doc += [t] * n
    documents.append(this_doc)
    if doc_n >= 25:
        break
                

Build a gensim dictionary and corpus and then train the model.

In [10]:
dictionary = gensim.corpora.Dictionary(documents)

In [11]:
dictionary.filter_extremes(no_below=10, no_above=0.5)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]


# train model, this might take some time
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=5,
    passes=15
)


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/).

In [12]:
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.092136 -0.038620       1        1  44.444767
2      0.074095 -0.047185       2        1  19.859734
3      0.055546  0.254724       3        1  16.909718
4      0.135468 -0.136876       4        1  12.996266
1     -0.357244 -0.032044       5        1   5.789517, topic_info=     Category         Freq         Term        Total  loglift  logprob
3098  Default  1676.000000         plus  1676.000000  30.0000  30.0000
4531  Default   607.000000         sans   607.000000  29.0000  29.0000
4460  Default   845.000000         dont   845.000000  28.0000  28.0000
4494  Default   876.000000      kennedy   876.000000  27.0000  27.0000
648   Default   661.000000       sloane   661.000000  26.0000  26.0000
2768  Default   318.000000        monde   318.000000  25.0000  25.0000
4560  Default   306.000000         vers   306.000000  24.0000  24.0000
24    Default   283.000000        amour   283.000000  23.0000  23.0000
2498  Default   234.000000         jour   234.000000  22.0000  22.0000
2019  Default   177.000000        femme   177.000000  21.0000  21.0000
4766  Default   263.000000  housekeeper   263.000000  20.0000  20.0000
3309  Default   271.000000         regt   271.000000  19.0000  19.0000
2169  Default   160.000000       grande   160.000000  18.0000  18.0000
2617  Default   153.000000         loin   153.000000  17.0000  17.0000
2210  Default   272.000000       hannah   272.000000  16.0000  16.0000
4465  Default   143.000000       esprit   143.000000  15.0000  15.0000
4459  Default   257.000000    donaldson   257.000000  14.0000  14.0000
4378  Default   282.000000         jeff   282.000000  13.0000  13.0000
4184  Default   134.000000       visage   134.000000  12.0000  12.0000
4549  Default   137.000000         tant   137.000000  11.0000  11.0000
4264  Default   369.000000     williams   369.000000  10.0000  10.0000
4742  Default   196.000000   fritzinger   196.000000   9.0000   9.0000
3036  Default   120.000000       petite   120.000000   8.0000   8.0000
3947  Default   674.000000         thee   674.000000   7.0000   7.0000
723   Default   407.000000      traubel   407.000000   6.0000   6.0000
4762  Default   206.000000        heyde   206.000000   5.0000   5.0000
2960  Default   207.000000       parlor   207.000000   4.0000   4.0000
3003  Default   111.000000      pendant   111.000000   3.0000   3.0000
773   Default   221.000000   wordsworth   221.000000   2.0000   2.0000
1887  Default   133.000000     ensemble   133.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
2498   Topic5   214.844757         jour   234.204010   2.7628  -4.0160
2032   Topic5    51.536789        fille    56.398178   2.7590  -5.4436
4561   Topic5    75.804893         veux    83.515442   2.7523  -5.0577
3036   Topic5   109.278992       petite   120.490219   2.7515  -4.6920
3003   Topic5    98.869217      pendant   111.408081   2.7297  -4.7921
4184   Topic5   119.226295       visage   134.360214   2.7296  -4.6049
4549   Topic5   121.414810         tant   137.557251   2.7243  -4.5867
4449   Topic5    55.924809        cours    63.960640   2.7149  -5.3619
4556   Topic5    75.181305      travail    86.537399   2.7084  -5.0660
24     Topic5   245.762833        amour   283.023773   2.7080  -3.8815
2169   Topic5   138.809204       grande   160.008728   2.7070  -4.4528
2617   Topic5   133.000748         loin   153.369446   2.7066  -4.4955
4516   Topic5    42.350716        pense    49.061108   2.7020  -5.6399
4498   Topic5    39.653618        libre    46.636089   2.6869  -5.7057
4509   Topic5    63.827881     nouvelle    75.177879   2.6855  -5.2297
3052   Topic5    36.958202         pied    43.617249   2.6835  -5.7761
4424   Topic5    57.594597         aura    69.163414   2.6661  -5.3325
4473   Topic5    34.714542      grandes    41.805965   2.6632  -5.8387
4500   Topic5    46.026920   